## Simple QnA Bot using Python and OpenAI API

In [1]:
from dotenv import load_dotenv
from openai import OpenAI
import os

In [ ]:
load_dotenv('config.env')
api_key = os.getenv('OPENAI_API_KEY')

client = OpenAI(api_key=api_key)

### Way 1 - Simple message field with system and user

In [ ]:
def llm_response(user_prompt):

    messages = [
        {
            "role": "system",
            "content": "You are a helpful QnA bot"
        },
        {
            "role": "user",
            "content": user_prompt
        }
    ]

    response = client.chat.completions.create(
        model = 'gpt-4o',
        messages = messages
    )

    return response.choices[0].message.content

In [ ]:
query = "What is the Capital of France?"
answer = llm_response(query)
print(answer)

The capital of France is Paris.


### Way 2 - using prompt engineering in separate functions

In [4]:
def bot_prompt(user_query):
    system_prompt = """
    You are an expert in answer user query about general world knowledge.
    Make sure the answers are short and crisp.
    Do not hallucinate. If you don't know the answer, just say - "I am not trained on this. I don't know the answer."
    """
    human_prompt = """
    Understand the below question and answer precisely.
    {0}
    """.format(user_query)

    return system_prompt, human_prompt

In [ ]:
def openai_response(system_prompt, human_prompt):
    messages = [
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": human_prompt
        }
    ]

    response = client.chat.completions.create(
        model = 'gpt-4o',
        messages = messages
    )

    return response.choices[0].message.content
    

In [ ]:
user_query = "Who is the CEO of Apple?"

In [5]:
system_prompt, human_prompt = bot_prompt(user_query)

In [ ]:
result = openai_response(system_prompt, human_prompt)
print(result)

### Adding Memory to the Bot
- Below code is to have a session memory (within a session) - not so recommended.
- Check persistent memory for accross sessions on next jupyter file.

In [1]:
conversation = []

def openai_response(system_prompt, human_prompt):

    # Initialize conversation only once at start
    if not conversation:  
        conversation.append({"role": "system", "content": system_prompt})
    
    # Add the new user message
    conversation.append({"role": "user", "content": human_prompt})

    # Send the full conversation history
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=conversation
    )

    reply = response.choices[0].message.content

    # Add assistant’s reply to memory
    conversation.append({"role": "assistant", "content": reply})

    return reply